In [2]:
import pandas as pd
import numpy as np

### Warnings
import warnings

### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

# For parallel processing
import multiprocessing

import torch.nn as nn
import torch
import torch.nn.functional as F

C:\Users\Wade\AppData\Local\Temp\ipykernel_10288\897370524.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv("data_iter1.csv")

C:\Users\Wade\AppData\Local\Temp\ipykernel_10288\633485919.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_iter1.csv")


In [4]:
from sklearn.metrics import mean_squared_error

ratings_matrix = pd.pivot_table(df.copy(), values='Rating', index='UserID', columns='MovieID')
ratings_matrix.head(2)

MovieID,1,2,3,4,5,6,7,8,10,11,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
normalized_ratings_matrix = ratings_matrix.subtract(ratings_matrix.mean(axis=1), axis=0)
normalized_ratings_matrix.head(2)

MovieID,1,2,3,4,5,6,7,8,10,11,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.816327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
similarity_matrix = ratings_matrix.T.corr()

In [7]:
similarity_matrix.head(5)

UserID,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.416667,-0.332182,0.333333,-0.172516,0.099340,NaN,-0.583333,3.305715e-01,-0.069171,...,-0.160128,0.755929,1.000000,NaN,0.381246,-0.248047,-4.517540e-01,NaN,1.838037e-01,-0.043519
2,0.416667,1.000000,0.236834,-0.270031,-0.208333,-0.061859,0.154303,0.024968,2.051123e-01,-0.054568,...,-0.709299,0.258341,0.368394,NaN,0.390907,0.363309,3.081451e-01,-0.500000,5.720776e-01,-0.012620
3,-0.332182,0.236834,1.000000,0.840168,-0.437621,-0.349215,0.790569,-0.528594,9.614813e-17,0.009305,...,-0.395285,-0.097590,-0.711556,NaN,0.427001,0.204619,-3.551092e-17,0.500000,3.092769e-01,-0.395285
4,0.333333,-0.270031,0.840168,1.000000,0.000000,NaN,-0.207514,0.534522,5.222330e-01,-0.212798,...,-0.645497,0.345857,-0.065606,NaN,-0.013558,0.445443,-1.132277e-01,-1.000000,3.845925e-16,0.509568
5,-0.172516,-0.208333,-0.437621,0.000000,1.000000,-0.592014,-0.028516,0.218130,-1.230573e-16,-0.023582,...,0.260875,0.707107,0.116775,0.5,0.548918,0.162048,-9.713482e-02,-0.928571,1.000000e+00,0.308273


In [8]:
df.shape

(994439, 35)

In [12]:
n_samples = int(0.01 * len(df))

# Group the DataFrame by user
grouped = df.copy().groupby('UserID', group_keys=False)

# Sample 20% of each user's ratings
test_df = grouped.apply(lambda x: x.sample(frac=0.01))
test_df.reset_index(drop=True, inplace=True)
test_df.head(2)

C:\Users\Wade\AppData\Local\Temp\ipykernel_10288\2758450849.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = grouped.apply(lambda x: x.sample(frac=0.01))


,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,1,1022,5,978300055,Cinderella (1950),Animation|Children's|Musical,F,1,10,48067,...,Cinderella has faith her dreams of a better li...,85.420,/avz6S9HYWs4O8Oe4PenBFNX4uDi.jpg,The greatest love story ever told.,"Family, Fantasy, Animation, Romance",Walt Disney Productions,United States of America,English,1950.0,Cinderella (1950)
1,2,2943,4,978298372,Indochine (1992),Drama|Romance,M,56,16,70072,...,Eliane Devries is the seemingly repressed owne...,14.948,/sAemRfr6mOSwtQ76e4yUQVImo8B.jpg,A great film from a mysterious world,"Drama, Romance","Investimage 3, Bac Films, Paradis Films, Orly ...",France,"French, Vietnamese",1992.0,Indochine (1992)


In [10]:
from concurrent.futures import ThreadPoolExecutor
def calculate_score_single(pair):
    u, i = pair
    print(f"Processing pair: User {u}, Item {i}")
    # Check whether the item is in the training dataset - ratings_matrix
    if i not in ratings_matrix.columns:
        return 2.5
    similarity_scores = similarity_matrix[u].drop(labels=u)
    normalized_ratings = normalized_ratings_matrix[i].drop(index=u)
    # Drop users that haven't rated the item
    similarity_scores.drop(index=normalized_ratings[normalized_ratings.isnull()].index, inplace=True)
    normalized_ratings.dropna(inplace=True)
    
    # If none of the other users have rated items in common with the user in question return the baseline value
    if similarity_scores.isna().all():
        return 2.5
    
    total_score = 0
    total_weight = 0
    for v in normalized_ratings.index:        
        # It's possible that another user rated the item but that
        # they have not rated any items in common with the user in question
        if not pd.isna(similarity_scores[v]):
            total_score += normalized_ratings[v] * similarity_scores[v]
            total_weight += abs(similarity_scores[v])
            
    avg_user_rating = ratings_matrix.T.mean()[u]
    
    return avg_user_rating + total_score / total_weight

def calculate_scores_parallel(user_item_pairs):
    results = []

    with ThreadPoolExecutor(max_workers=12) as executor:  # Limit to 5 concurrent workers
        futures = [executor.submit(calculate_score_single, pair) for pair in user_item_pairs]
        for future in futures:
            results.append(future.result())
    
    return np.array(results)

In [13]:
test_df.shape

(9646, 35)

This is just to check the accuracy

In [14]:
test_ratings = np.array(test_df["Rating"])
user_item_pairs_list = list(zip(test_df["UserID"], test_df["MovieID"]))
pred_ratings = calculate_scores_parallel(user_item_pairs_list)

Processing pair: User 1, Item 1022
Processing pair: User 2, Item 2943
Processing pair: User 3, Item 2617
Processing pair: User 5, Item 2683
Processing pair: User 5, Item 3578
Processing pair: User 6, Item 1380
Processing pair: User 8, Item 58
Processing pair: User 9, Item 3510
Processing pair: User 10, Item 1221
Processing pair: User 10, Item 1104
Processing pair: User 10, Item 916
Processing pair: User 10, Item 3198
Processing pair: User 11, Item 3755
Processing pair: User 13, Item 50
Processing pair: User 15, Item 1370
Processing pair: User 15, Item 3723
Processing pair: User 17, Item 3700
Processing pair: User 17, Item 1356
Processing pair: User 18, Item 50
Processing pair: User 18, Item 2025
Processing pair: User 18, Item 2
Processing pair: User 19, Item 1372
Processing pair: User 19, Item 198
Processing pair: User 19, Item 1327
Processing pair: User 22, Item 2763
Processing pair: User 22, Item 786
Processing pair: User 22, Item 3020
Processing pair: User 23, Item 1306
Processing p

In [15]:
print(np.sqrt(mean_squared_error(test_ratings, pred_ratings)))

0.812441705018607


In [16]:
pred_ratings

array([4.54789803, 3.77899766, 3.27519197, ..., 4.26807258, 3.77609391,
       3.72328814])

In [31]:
import numpy as np

def calculate_all_ratings_vectorized(similarity_matrix, normalized_ratings_matrix):
    similarity_scores = similarity_matrix.values  # Convert similarity matrix to NumPy array
    normalized_ratings = normalized_ratings_matrix.values  # Convert normalized ratings matrix to NumPy array

    # Transpose similarity_scores to match the shape of normalized_ratings for broadcasting
    similarity_scores_transposed = similarity_scores.T

    # Calculate the dot product of similarity scores and normalized ratings
    dot_product = np.dot(similarity_scores_transposed, normalized_ratings)

    # Calculate the total weight for each user
    total_weight = np.abs(similarity_scores).sum(axis=1)

    # Calculate the average user rating
    avg_user_rating = np.nanmean(normalized_ratings, axis=1)

    # Calculate the predicted ratings for all users and items
    predicted_ratings = avg_user_rating[:, np.newaxis] + (dot_product / total_weight[:, np.newaxis])

    return predicted_ratings

# Assuming you have similarity_matrix and normalized_ratings_matrix ready
predicted_ratings = calculate_all_ratings_vectorized(similarity_matrix.copy().fillna(0), normalized_ratings_matrix.copy().fillna(0))

C:\Users\Wade\AppData\Local\Temp\ipykernel_10288\2754063641.py:20: RuntimeWarning: invalid value encountered in divide
  predicted_ratings = avg_user_rating[:, np.newaxis] + (dot_product / total_weight[:, np.newaxis])


In [34]:
predicted_rating_final = pd.DataFrame(predicted_ratings, columns = normalized_ratings_matrix.columns.values, index = normalized_ratings_matrix.index.values)
predicted_rating_final.head(2)

,1,2,3,4,5,6,7,8,10,11,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,0.105734,-0.002382,0.003538,-0.002275,-0.004631,0.013552,-0.000152,-0.000497,0.003449,0.006636,...,-0.005578,-0.000076,-0.006809,-0.003438,-0.000480,0.000845,0.005917,0.00450,0.000355,0.005915
2,0.121629,-0.024133,-0.021284,-0.011096,-0.012038,0.031362,-0.008046,-0.004800,0.000241,0.029030,...,-0.006026,-0.001080,-0.006005,-0.017149,0.000287,0.007871,0.013477,0.00192,0.001228,0.007322


In [44]:
# Assuming predicted_rating_final is a DataFrame where rows represent users and columns represent movies
first_10_users_ratings = predicted_rating_final.iloc[:10]

# Extracting movie IDs from the column indices of the DataFrame
movie_ids = predicted_rating_final.columns

import pandas as pd

# Initialize a list to store top 5 movies for each user
top_movies_data = []

# Iterate over the first 10 users and their ratings
for user_id, user_ratings in first_10_users_ratings.iterrows():
    # Sort the predicted ratings for the user
    sorted_ratings = sorted(zip(movie_ids, user_ratings), key=lambda x: x[1], reverse=True)
    top_5_movies = sorted_ratings[:5]
    
    # Append the top 5 movies for the user to the list
    for movie_id, rating in top_5_movies:
        top_movies_data.append({'UserID': user_id, 'MovieID': movie_id, 'PredictedRating': rating})

# Create a DataFrame from the list
top_movies_df_first10user = pd.DataFrame(top_movies_data)
top_movies_df_first10user.head(2)


,UserID,MovieID,PredictedRating
0,1,2028,0.214227
1,1,527,0.185960


In [39]:
# Create a dictionary to store movie ID-title pairs
movie_id_title_dict = {}

# Iterate over the DataFrame to populate the dictionary
for index, row in df.iterrows():
    movie_id = row['MovieID']
    title = row['Title']
    movie_id_title_dict[movie_id] = title

# Now movie_id_title_dict contains the mapping of movie IDs to titles
print(movie_id_title_dict)


{1193: "One Flew Over the Cuckoo's Nest (1975)", 661: 'James and the Giant Peach (1996)', 914: 'My Fair Lady (1964)', 3408: 'Erin Brockovich (2000)', 2355: "A Bug's Life (1998)", 1197: 'The Princess Bride (1987)', 1287: 'Ben-Hur (1959)', 2804: 'A Christmas Story (1983)', 594: 'Snow White and the Seven Dwarfs (1937)', 919: 'The Wizard of Oz (1939)', 595: 'Beauty and the Beast (1991)', 938: 'Gigi (1958)', 2398: 'Miracle on 34th Street (1947)', 2918: "Ferris Bueller's Day Off (1986)", 1035: 'The Sound of Music (1965)', 2791: 'Airplane! (1980)', 2687: 'Tarzan (1999)', 2018: 'Bambi (1942)', 3105: 'Awakenings (1990)', 2797: 'Big (1988)', 2321: 'Pleasantville (1998)', 720: 'Wallace & Gromit: The Best of Aardman Animation (1996)', 1270: 'Back to the Future (1985)', 527: "Schindler's List (1993)", 2340: 'Meet Joe Black (1998)', 1097: 'E.T. the Extra-Terrestrial (1982)', 1721: 'Titanic (1997)', 1545: 'Ponette (1996)', 745: 'A Close Shave (1995)', 2294: 'Antz (1998)', 3186: 'Girl, Interrupted (19

In [47]:
top_movies_df_first10user['Title'] = top_movies_df_first10user['MovieID'].map(movie_id_title_dict)

# Display the DataFrame with Movie titles
top_movies_df_first10user

,UserID,MovieID,PredictedRating,Title
0,1,2028,0.214227,Saving Private Ryan (1998)
1,1,527,0.185960,Schindler's List (1993)
2,1,1270,0.142985,Back to the Future (1985)
3,1,1193,0.111870,One Flew Over the Cuckoo's Nest (1975)
4,1,1,0.105734,Toy Story (1995)
5,2,1196,0.275901,Star Wars: Episode V - The Empire Strikes Back...
6,2,260,0.273874,Star Wars: Episode IV - A New Hope (1977)
7,2,593,0.262619,The Silence of the Lambs (1991)
8,2,318,0.255912,The Shawshank Redemption (1994)
9,2,110,0.255131,Braveheart (1995)
